In [ ]:
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
from pathlib import Path

In [ ]:
from ModelSetting import Model
from UtilityFunctions import U_SMOCU
from UtilityFunctions import U_BALD, U_MES
from OptimizeMethod import MCSelector
from OptimizeMethod import SGD
from OptimizeMethod import MCSelector_2
from OptimizeMethod import Multi_start_SGD
from OptimizeMethod import hybrid_entropy_smocu_step

In [ ]:
from Active_Learning import plot_gp_decision_boundary, plot_gp_entropy_field, plot_acquisition_history, plot_acquisition_mc_field, plot_gp_prob_variance_field

In [ ]:
from Active_Learning import plot_gp_latent_variance_field, plot_gp_latent_mean_field

In [ ]:
def relative_sequential_difference(a):
    """
    Compute relative absolute differences between sequential elements.

    Parameters
    ----------
    a : array-like
        Input array.

    Returns
    -------
    r : np.ndarray
        Relative differences, length len(a) - 1.
    """
    a = np.asarray(a, dtype=float)

    numerator = np.abs(np.diff(a))
    denominator = np.max(a)

    return numerator / (denominator)

In [ ]:
def label_nonlinear_boundary(x1, x2):
    """
    2D nonlinear trial funcion.

    Returns
    -------
    labels : int or array of integers
        0 if the point is above the curve. 
        1 if the point is on/below the curve. 
    """
    x1 = np.asarray(x1, dtype=float)
    x2 = np.asarray(x2, dtype=float)

    boundary = 0.2 + 0.6 * x1**2
    labels = np.where(x2 > boundary, 0, 1)
    return labels

def label_linear_sinus_boundary(x1, x2):
    """
    2D linear sinus boundary

    Returns
    -------
    labels : int or array of integers
        0 if the point is above the curve. 
        1 if the point is on/below the curve.
    """
    x1 = np.asarray(x1, dtype=float)
    x2 = np.asarray(x2, dtype=float)

    a = 0.2  # overall amplitude factor; adjust if you want stronger/weaker wiggles
    boundary = x1 + a * np.sin(7.0 * np.pi * x1)

    labels = np.where(x2 > boundary, 0, 1)
    return labels

def label_parabola(x1, x2):
    """
    2D parabola boundary. 

    Returns
    -------
    labels : int or array of integers
        0 if the point is above the curve. 
        1 if the point is on/below the curve. 
    """
    x1 = np.asarray(x1, dtype=float)
    x2 = np.asarray(x2, dtype=float)

    boundary = 0.2 + 0.6 * 7*(x1-0.5)**2
    labels = np.where(x2 > boundary, 0, 1)
    return labels

def label_sqrt_boundary(x1, x2):
    x1 = np.asarray(x1, dtype=float)
    x2 = np.asarray(x2, dtype=float)

    boundary = 0.5 * np.sqrt(np.clip(x1 - 0.2, 0.0, None))

    labels = np.where(x2 > boundary, 0, 1)
    return labels

def label_vertical_boundary(x1, x2):
    x1 = np.asarray(x1, dtype=float)
    x2 = np.asarray(x2, dtype=float)

    boundary = 0.8

    labels = np.where(x1 > boundary, 1, 0)
    return labels

def label_nonlinear_boundary_3d(x1, x2, x3):
    """
    3D nonlinear decision boundary.

    Labels:
        0 if the point is above the surface. 
        1 if the point is on/below the surface. 
    """
    x1 = np.asarray(x1, dtype=float)
    x2 = np.asarray(x2, dtype=float)
    x3 = np.asarray(x3, dtype=float)

    boundary = 0.2 + 0.3 * (x1**2 + x2**2)
    labels = np.where(x3 > boundary, 0, 1)
    return labels

def label_nonlinear_boundary_4d(x1, x2, x3, x4):
    """
    4D nonlinear decision boundary.

    Returns
    -------
    labels : int or array of integers. 
        0 if x4 is above the surface. 
        1 if x4 is on/below the surface. 
    """
    x1 = np.asarray(x1, dtype=float)
    x2 = np.asarray(x2, dtype=float)
    x3 = np.asarray(x3, dtype=float)
    x4 = np.asarray(x4, dtype=float)

    boundary = 0.25 + 0.35 * x1**2 + 0.20 * x2 + 0.10 * np.sin(2 * np.pi * x3)
    labels = np.where(x4 > boundary, 0, 1)
    return labels


def label_nonlinear_boundary_3d_sinus(x1, x2, x3):
    """
    3D nonlinear sinus decision boundary. 

    Labels:
        0 if the point is above the surface. 
        1 if the point is on/below the surface. 
    """
    x1 = np.asarray(x1, dtype=float)
    x2 = np.asarray(x2, dtype=float)
    x3 = np.asarray(x3, dtype=float)

    boundary = (
        0.2
        + 0.3 * (x1**2 + x2**2)
        + 0.1 * np.sin(2 * np.pi * x1)
    )
    labels = np.where(x3 > boundary, 0, 1)
    return labels

def label_toy_6d(x1, x2, x3, x4, x5, x6):
    """
    6D decision boundary. 

    Returns:
    --------
    Labels:
        0 if x6 is above boundary. 
        1 if x6 is on/below the boundary. 
    """
    x1 = np.asarray(x1, dtype=float)
    x2 = np.asarray(x2, dtype=float)
    x3 = np.asarray(x3, dtype=float)
    x4 = np.asarray(x4, dtype=float)
    x5 = np.asarray(x5, dtype=float)
    x6 = np.asarray(x6, dtype=float)

    boundary = (
        0.25
        + 0.20 * x1**2
        + 0.10 * x2
        + 0.15 * x3**2
        + 0.10 * x4**2 * x5
        + 0.05 * x1 * x3**2
        - 0.05 * x2 * x4
    )

    labels = np.where(x6 > boundary, 0, 1)
    return labels

def label_toy_12d(
    x1, x2, x3, x4, x5, x6,
    x7, x8, x9, x10, x11, x12
):
    """
    12D decision boundary. 

    Labels:
        0 if x12 above boundary. 
        1 if x12 is on/below boundary. 
    """

    x1  = np.asarray(x1, dtype=float)
    x2  = np.asarray(x2, dtype=float)
    x3  = np.asarray(x3, dtype=float)
    x4  = np.asarray(x4, dtype=float)
    x5  = np.asarray(x5, dtype=float)
    x6  = np.asarray(x6, dtype=float)
    x7  = np.asarray(x7, dtype=float)
    x8  = np.asarray(x8, dtype=float)
    x9  = np.asarray(x9, dtype=float)
    x10 = np.asarray(x10, dtype=float)
    x11 = np.asarray(x11, dtype=float)
    x12 = np.asarray(x12, dtype=float)

    boundary = (
        0.30
        + 0.12 * x1**2
        + 0.08 * x2
        + 0.10 * x3**2
        + 0.06 * x4 * x5
        + 0.07 * x6**2
        + 0.05 * x7 * x8
        + 0.06 * x9**2
        + 0.04 * x10 * x11
        + 0.03 * x1 * x3
        - 0.04 * x2 * x4
        + 0.02 * x5 * x6**2
    )

    labels = np.where(x12 > boundary, 0, 1)
    return labels

In [ ]:
# ---- 2D      initial design: one point in normalised space ----

# x0_norm = np.array([[0.8, 0.1]])
x0_norm = np.array([[0.9, 0.1]])
# y0 = label_sqrt_boundary(x0_norm[0, 0], x0_norm[0, 1])
y0 = label_vertical_boundary(x0_norm[0, 0], x0_norm[0, 1])

# y0 = int(1)

X_init = x0_norm                    # shape (1, 2)
Y_init = np.array([[y0]], float)    # shape (1, 1)

model = Model(X_init, Y_init, optimize=False, mean_fix=True)

In [ ]:
# plot_gp_decision_boundary(
#         model,
#         X_init, Y_init.ravel(),
#         title=f"GPC Decision Boundary - Initial",
#         save_path=Path(base)/f'GPC_Decision_Boundary_variance10_mean2_ls0.1.pdf',
#         show=True,
#     )

In [ ]:
model.predict_proba(np.array([[0.01, 0.99]]))

In [ ]:
plot_gp_latent_mean_field(
    model,
    X_init, 
    Y_init, 
    title="Latent mean – initial",
    # save_path=f'GPC_latentmean_variance10_mean2_ls0.1.pdf',
)

In [ ]:
# plot_gp_latent_variance_field(
#     model,
#     X_init, 
#     Y_init, 
#     title="Latent variance – initial",
#     save_path=Path(base)/f'GPC_latentvariance_variance10_mean2_ls0.1.pdf',
# )

In [ ]:
# # ---- 3D       initial design: one point in normalised space ----
# x0_norm = np.array([[0.8, 0.8, 0.1]])
# y0 = label_nonlinear_boundary_3d(x0_norm[0, 0], x0_norm[0, 1], x0_norm[0, 2])

# X_init = x0_norm                    # shape (1, 2)
# Y_init = np.array([[y0]], float)    # shape (1, 1)

# model = Model(X_init, Y_init, optimize=False, mean_fix=True)

In [ ]:
# # ---- 4D       initial design: one point in normalised space ----
# x0_norm = np.array([[0.0, 0.0, 0.0, 0.1]])
# y0 = label_nonlinear_boundary_4d(x0_norm[0, 0], x0_norm[0, 1], x0_norm[0, 2], x0_norm[0, 3])

# X_init = x0_norm                    # shape (1, 2)
# Y_init = np.array([[y0]], float)    # shape (1, 1)

# model = Model(X_init, Y_init, optimize=False, mean_fix=True)

In [ ]:
# # ---- 6D       initial design: one point in normalised space ----
# x0_norm = np.array([[0.5, 0.5, 0.5, 0.5, 0.5, 0.1]])
# y0 = label_toy_6d(x0_norm[0, 0], x0_norm[0, 1], x0_norm[0, 2], x0_norm[0, 3], x0_norm[0, 4], x0_norm[0, 5])

# X_init = x0_norm                    # shape (1, 2)
# Y_init = np.array([[y0]], float)    # shape (1, 1)

# model = Model(X_init, Y_init, optimize=False, mean_fix=True)

In [ ]:
# # ---- 12D       initial design: one point in normalised space ----
# x0_norm = np.array([[0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.10]])
# y0 = label_toy_12d(x0_norm[0, 0], x0_norm[0, 1], x0_norm[0, 2], x0_norm[0, 3], x0_norm[0, 4], x0_norm[0, 5], x0_norm[0, 6], x0_norm[0, 7], x0_norm[0, 8], x0_norm[0, 9], x0_norm[0, 10], x0_norm[0, 11])

# X_init = x0_norm                    # shape (1, 2)
# Y_init = np.array([[y0]], float)    # shape (1, 1)

# model = Model(X_init, Y_init, optimize=False, mean_fix=True)

In [ ]:
# ---- NR-SMOCU acquisition ----

smocu_x_num = 2000      # number of points to approximate SMOCU integral
mc_search_num = 2_000    # number of candidate points for MC search
SGD_steps = 200

acq = U_SMOCU(
    softtype=2,         # soft MOCU
    k=20,
    x_num=smocu_x_num,
    approx_label=True   # NR-SMOCU (no-retraining)
)

In [ ]:
n_iterations = 100

X_history = [X_init.copy()]
Y_history = [Y_init.copy()]

# base = r""
# base_path = Path(base)
acq_list = []
log_likelihood_list = []
miscl_list = []
start_opt = False
count = 0
convergence_count = 0
optimization_count = 0

do_smocu_sgd = True

In [ ]:
H_mean_list = []
H_med_list = []
boundary_frac_list = []
flips_frac_list = []

In [ ]:
from scipy.stats import qmc

d = 2  # dimension
n_monitor = 20_000  # or 1000 if you can afford it

sampler = qmc.Sobol(d=d, scramble=True, seed=123)
X_monitor = sampler.random(n_monitor)

p_monitor = model.predict_proba(X_monitor)[:, 1]
# Entropy
eps = 1e-12
H = -p_monitor * np.log(p_monitor + eps) - (1-p_monitor) * np.log(1-p_monitor + eps)
H_mean = H.mean()
H_med  = np.median(H)

# Boundary spread
boundary_mask = (p_monitor > 0.3) & (p_monitor < 0.7)
boundary_frac = boundary_mask.mean()   # in [0,1]

# Boundary change initialisation
y_mon_now = (p_monitor >= 0.5).astype(int)

H_mean_list.append(H_mean)
H_med_list.append(H_med)
boundary_frac_list.append(boundary_frac)

flips_frac_count = 0
y_mon_prev = y_mon_now.copy()

In [ ]:
# Active learning loop
# ----------------------------------------
for it in range(n_iterations):
    it_print = it + 1
    print(f"\n=== Iteration {it_print} ===")
    
    print("Mean function:", model.gpc.mean_function)

    # x_star_norm, acq_value = hybrid_entropy_smocu_step(
    #     model=model,
    #     acq_smocu=acq,
    #     sobol_num=mc_search_num,
    #     frac_keep_entropy=0.00002,
    #     n_entropy_steps=80,
    #     entropy_lr=1e-5,
    #     p_band=(0.4, 0.6),
    #     do_smocu_sgd=do_smocu_sgd,
    #     smocu_n_steps=200,
    #     smocu_lr=1e-3,
    #     sobol_seed=None,
    # )
    
    
    x_star_norm, acq_value = Multi_start_SGD(
        acq,
        model=model,
        mc_search_num=mc_search_num,
        learning_rate=0.001,
        n_starts=1,
        top_frac=0.01,
        n_sgd_steps=SGD_steps,
        return_field=False,
    )

    acq_list.append(acq_value)
    acq_arr = np.array(acq_list)
    
    acq_rel = relative_sequential_difference(acq_arr)
    
    # -------------------- 2d -------------------------- #
    # y_star = label_nonlinear_boundary(x_star_norm[0], x_star_norm[1])
    # y_star = label_linear_sinus_boundary(x_star_norm[0], x_star_norm[1])
    # y_star = label_parabola(x_star_norm[0], x_star_norm[1])
    # y_star = label_sqrt_boundary(x_star_norm[0], x_star_norm[1])
    y_star = label_vertical_boundary(x_star_norm[0], x_star_norm[1])


    # ------------------ 3d ----------------------------#
    # y_star = label_nonlinear_boundary_3d(x_star_norm[0], x_star_norm[1], x_star_norm[2])
    # y_star = label_nonlinear_boundary_3d_sinus(x_star_norm[0], x_star_norm[1], x_star_norm[2])

    # ------------------ 4d --------------------------#
    # y_star = label_nonlinear_boundary_4d(x_star_norm[0], x_star_norm[1], x_star_norm[2], x_star_norm[3])

    # ------------------ 5d --------------------------#

    # ------------------ 6d --------------------------#
    # y_star = label_toy_6d(x_star_norm[0], x_star_norm[1], x_star_norm[2], x_star_norm[3], x_star_norm[4], x_star_norm[5])

    # ---------------- 12d -----------------------------# 
    # y_star = label_toy_12d(x_star_norm[0], x_star_norm[1], x_star_norm[2], x_star_norm[3], x_star_norm[4], x_star_norm[5], x_star_norm[6], x_star_norm[7], x_star_norm[8], x_star_norm[9], x_star_norm[10], x_star_norm[11])

    # Administration
    X_history.append(np.vstack([X_history[-1], x_star_norm]))
    Y_history.append(np.vstack([Y_history[-1], [[y_star]]]))

    X_current = X_history[-1]
    y_current = Y_history[-1].ravel().astype(int)
    
    lengthscales_before = model.gpc.kern.lengthscale
    
    do_opt = False
    # if len(flips_frac_list) > 10:
    #     if np.all(np.array(flips_frac_list[-10:]) <= np.max(np.array(flips_frac_list[1:])) * 0.25):
    #         start_opt = True
    #     if count % 10 == 0 and start_opt:
    #         do_opt = True
    #     if start_opt:
    #         count += 1

        
    if flips_frac_count > 20:
        start_opt = True
        # do_smocu_sgd = False
        if count % 10 == 0 and start_opt:
            do_opt = True
        # if flips_frac_count > 20 and flips_frac_count < 22:
        #     do_opt = True
        if start_opt:
            count += 1
    
    model.Update(x_star_norm, y_star, optimize=do_opt, kern_variance_fix=False, mean_fix=True, ll_tol = 0.0)
    # p_infty = 0.1587
    # if start_opt:
    #     p_infty = 0.01
    # model.Update(x_star_norm, y_star, optimize=do_opt, kern_variance_fix=False, mean_fix=True, ll_tol = 0.0, p_infty=p_infty)
    

    log_likelihood = model.gpc.log_likelihood()
    log_likelihood_list.append(log_likelihood)

    lengthscales_after = model.gpc.kern.lengthscale

    print("Next normalised point:", x_star_norm)
    print("Label (failure?):", y_star, "   Acquisition:", acq_value)
    print("Kernel variance:", model.gpc.kern.variance)
    print("Kernel lengthscales:", model.gpc.kern.lengthscale)
    print("Mean function:", model.gpc.mean_function)
    print("Log_Likelihood:", log_likelihood)
    if len(acq_rel) > 10:
        print('Relative acquisition values:', acq_rel[-11:])
    print("Count from moment of optimization:", count)
    
    miscl = model.training_misclassification(X_current, y_current)
    miscl_list.append(miscl)
    miscl_arr = np.array(miscl_list)
    print('Misclassification mertic:', miscl)

    if np.any(lengthscales_before != lengthscales_after):
        optimization_count += 1
    print("Optimzation Count:", optimization_count)


    p_monitor = model.predict_proba(X_monitor)[:, 1]  # P(y=1|x)
    # Entropy
    eps = 1e-12
    H = -p_monitor * np.log(p_monitor + eps) - (1-p_monitor) * np.log(1-p_monitor + eps)
    H_mean = H.mean()
    H_med  = np.median(H)
    
    # Boundary spread
    boundary_mask = (p_monitor > 0.3) & (p_monitor < 0.7)
    boundary_frac = boundary_mask.mean()   # in [0,1]
    
    # Boundary change
    y_mon_now = (p_monitor >= 0.5).astype(int)   
    
    flips_frac = np.mean(y_mon_now != y_mon_prev)
    y_mon_prev = y_mon_now.copy()

    H_mean_list.append(H_mean)
    H_med_list.append(H_med)
    boundary_frac_list.append(boundary_frac)
    flips_frac_list.append(flips_frac)


    if flips_frac < 0.25 * np.max(flips_frac_list):
        flips_frac_count += 1

    print("Flips_frac_count:", flips_frac_count)
    print("Boundary spread metric:", boundary_frac)
    diff_boundary_frac = np.diff(np.array(boundary_frac_list))

    
    # Plotting for two-dimensional problems
    plot_gp_decision_boundary(
        model,
        X_current, y_current,
        title=f"GPC Decision Boundary – iteration {it_print}",
        # save_path=Path(base)/'Prediction_Field'/f'GPC_Decision_Boundary_it_{it_print}.pdf',
        show=True,
    )

    plot_gp_latent_mean_field(
        model,
        X_current, 
        y_current, 
        title=f"GPC Latent Mean – iteration {it_print}",
        # save_path=Path(base)/"Latent_Mean"/ f'GPC_latentmean_it_{it_print}.pdf',
        show=False,
    )

    plot_gp_latent_variance_field(
        model,
        X_current, 
        y_current, 
        title=f"GPC Latent Variance – iteration {it_print}",
        # save_path=Path(base)/"Latent_Variance"/ f'GPC_latentvariance_it_{it_print}.pdf',
        show=False,
    )
    
    # plot_gp_entropy_field(
    #     model,
    #     X_current, y_current,
    #     title=f"GPC Entropy – iteration {it_print}", 
    #     # save_path=Path(base)/'Figures'/f'GPC_Entropy_Field_it_{it_print}.png',
    #     show=True,
    # )

    # plot_acquisition_mc_field(
    #     xspace,
    #     utilitymat,
    #     x_star=x_star_norm_1,
    #     x_star_sgd =x_star_norm,
    #     X_history=X_history[-1],
    #     title=f"MC acquisition field – iteration {it_print}",
    #     # save_path=Path(base)/'Figures'/f'Acq_MC_it_{it_print}.png',
    #     show=True,
    # )

    # plot_acquisition_history(
    #     acq_history,
    #     Path(base)/'Figures'/ "Acquisition_vs_iterations.png"
    # )

    # Stopping rule
    if count > 1 and boundary_frac_list[-1] < 0.01 and np.all(miscl_arr[-11:] < 0.05):
        print(
            f"\nStopping early at iteration {it_print}: "
            f"Boundary band < 0.01."
        )
        break

    # if count > 1 and boundary_frac_list[-1] < 0.05 and np.all(miscl_arr[-11:] < 0.05) and np.all(diff_boundary_frac[-21:] < 0.0005):
    #     print(
    #         f"\nStopping early at iteration {it_print}: "
    #         f"Boundary band < 0.05 and no improvement anymore on boundary metric."
    #     )
    #     break

X_final = X_history[-1]
Y_final = Y_history[-1]
print("\nTotal evaluated points:", X_final.shape[0])

In [ ]:
savepath = Path(base) / "Monitor_Figures"

In [ ]:
iterations = np.arange(1, len(miscl_list)+1)
plt.plot(iterations, miscl_list)
plt.hlines(0.05, 0, len(miscl_list), color='r', ls='--', label='Misclassification threshold: 0.05')
plt.title("Misclassification ratio")
plt.ylabel("Ratio [-]")
plt.xlabel("Iterations [-]")
plt.grid(alpha=0.3)
plt.legend()
# plt.savefig(Path(savepath) / 'Misclassification_ratio.pdf')

In [ ]:
iterations = np.arange(1, len(log_likelihood_list)+1)
plt.plot(iterations, log_likelihood_list)
plt.title("Log_Likelihood")
plt.xlabel("iterations [-]")
plt.ylabel("Log likelihood [-]")
plt.grid(alpha=0.3)
# plt.savefig(Path(savepath) / 'Log_Likelihood.pdf')

In [ ]:
iterations = np.arange(1, len(acq_arr)+1)
plt.plot(iterations, acq_arr)
plt.title("Acquisition values")
plt.ylabel("Acquisition values [-]")
plt.xlabel("Iterations [-]")
plt.grid(alpha=0.3)
# plt.ylim(0, 0.002)
# plt.savefig(Path(savepath) / 'Acquisition_values.pdf')

In [ ]:
r = relative_sequential_difference(acq_arr*100000)
plt.plot(np.arange(1, len(r)+1), r)
# plt.hlines(0.03, 0, len(r), ls='--', color='r', label="Hyperparameter optimization threshold: 0.03")
# plt.vlines(44, 0, 0.4, ls='--', color='k')
# plt.hlines(0.02, 0, len(r), ls='--', color='b', label="Convergence threshold: 0.02")
plt.title("Change in relative acquisition values")
plt.ylabel("Relative acquisition change [-]")
plt.xlabel("Iterations [-]")
plt.grid(alpha=0.3)
plt.legend()
# plt.savefig(Path(savepath) / 'change_in_relative_acquisition_values.pdf')

In [ ]:
model.gpc.kern

In [ ]:
model.gpc.kern.lengthscale

In [ ]:
Y_current_new = y_current.astype(float).reshape(-1, 1)  # shape (61, 1)
model2 = model.ModelTrain(X_current, Y_current_new, optimize=True, kern_lengthscale_fix=False, kern_variance_fix=False, mean_fix=True)
model2.gpc.kern

In [ ]:
plot_gp_decision_boundary(
        model2,
        X_current, y_current,
        title=f"GPC Decision Boundary – Final",
        # save_path=Path(base) / f'GPC_Decision_Boundary_final.pdf',
        show=True,
    )

In [ ]:
model2.gpc.kern.lengthscale

In [ ]:
model2.gpc.mean_function

In [ ]:
# # Plotting function for 3D sinus boundary problem
# # ---------------------------------------------------- #

# import numpy as np
# import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D

# def plot_gp_decision_boundary_3d_sinus(
#     model,
#     X,
#     y,
#     title="GPC Decision Boundary – 3D (sinus toy)",
#     n_grid=25,
#     eps=0.03,
#     show_true_boundary=True,
#     show=True,
# ):
#     X = np.asarray(X, float).reshape(-1, 3)
#     y = np.asarray(y, int).reshape(-1)

#     assert getattr(model, "f_num", 3) == 3, "plot_gp_decision_boundary_3d_sinus assumes f_num == 3"

#     fig = plt.figure(figsize=(8, 6))
#     ax = fig.add_subplot(111, projection="3d")

#     mask0 = (y == 0)
#     mask1 = (y == 1)

#     ax.scatter(
#         X[mask0, 0], X[mask0, 1], X[mask0, 2],
#         c="blue", label="class 0", alpha=0.8
#     )
#     ax.scatter(
#         X[mask1, 0], X[mask1, 1], X[mask1, 2],
#         c="red", label="class 1", alpha=0.8
#     )

#     grid = np.linspace(0.0, 1.0, n_grid)
#     Xg, Yg, Zg = np.meshgrid(grid, grid, grid, indexing="ij")
#     pts = np.column_stack([Xg.ravel(), Yg.ravel(), Zg.ravel()])

#     p = model.predict_proba(pts)[:, 1]  

#     mask_bd = np.abs(p - 0.5) < eps
#     bd_pts = pts[mask_bd]

#     if bd_pts.size > 0:
#         ax.scatter(
#             bd_pts[:, 0], bd_pts[:, 1], bd_pts[:, 2],
#             s=5, c="k", alpha=0.4, label="GP boundary (p≈0.5)"
#         )

#     if show_true_boundary:
#         u = np.linspace(0.0, 1.0, n_grid)
#         v = np.linspace(0.0, 1.0, n_grid)
#         U, V = np.meshgrid(u, v, indexing="ij")

#         W = 0.2 + 0.3 * (U**2 + V**2) + 0.1 * np.sin(2 * np.pi * U)

#         ax.plot_surface(
#             U, V, W,
#             alpha=0.3,
#             linewidth=0,
#             antialiased=True,
#             color="gray",
#         )
#         from matplotlib.lines import Line2D
#         proxy_true = Line2D([0], [0], color="gray", lw=6, alpha=0.3)
#         handles, labels = ax.get_legend_handles_labels()
#         handles.append(proxy_true)
#         labels.append("true boundary")
#         ax.legend(handles, labels)
#     else:
#         ax.legend()

#     ax.set_xlabel("x1")
#     ax.set_ylabel("x2")
#     ax.set_zlabel("x3")
#     ax.set_title(title)

#     ax.set_xlim(0, 1)
#     ax.set_ylim(0, 1)
#     ax.set_zlim(0, 1)

#     if show:
#         plt.tight_layout()

In [ ]:
# plot_gp_decision_boundary_3d_sinus(
#     model,
#     X_current,
#     y_current,
#     title=f"GPC Decision Boundary – 3D sinus – iteration {it_print}",
#     n_grid=100,
#     eps=0.03,
#     show_true_boundary=True,
#     show=True,
# )
# savepath = r""
# # plt.savefig(Path(savepath) / 'Decision_boundary_2.png')

In [ ]:
# # Plotting function for 3D nonlinear boundary problem
# # ---------------------------------------------------- #

# import numpy as np
# import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D

# def plot_gp_decision_boundary_3d(
#     model,
#     X,
#     y,
#     title="GPC Decision Boundary – 3D",
#     n_grid=25,
#     eps=0.03,
#     show_true_boundary=True,
#     show=True,
# ):
#     X = np.asarray(X, float).reshape(-1, 3)
#     y = np.asarray(y, int).reshape(-1)

#     assert getattr(model, "f_num", 3) == 3, "plot_gp_decision_boundary_3d assumes f_num == 3"

#     fig = plt.figure(figsize=(8, 6))
#     ax = fig.add_subplot(111, projection="3d")

#     mask0 = (y == 0)
#     mask1 = (y == 1)

#     ax.scatter(
#         X[mask0, 0], X[mask0, 1], X[mask0, 2],
#         c="blue", label="class 0", alpha=0.8
#     )
#     ax.scatter(
#         X[mask1, 0], X[mask1, 1], X[mask1, 2],
#         c="red", label="class 1", alpha=0.8
#     )

#     grid = np.linspace(0.0, 1.0, n_grid)
#     Xg, Yg, Zg = np.meshgrid(grid, grid, grid, indexing="ij")
#     pts = np.column_stack([Xg.ravel(), Yg.ravel(), Zg.ravel()])

#     p = model.predict_proba(pts)[:, 1]

#     mask_bd = np.abs(p - 0.5) < eps
#     bd_pts = pts[mask_bd]

#     if bd_pts.size > 0:
#         ax.scatter(
#             bd_pts[:, 0], bd_pts[:, 1], bd_pts[:, 2],
#             s=5, c="k", alpha=0.4, label="GP boundary (p≈0.5)"
#         )

#     if show_true_boundary:
#         u = np.linspace(0.0, 1.0, n_grid)
#         v = np.linspace(0.0, 1.0, n_grid)
#         U, V = np.meshgrid(u, v, indexing="ij")
#         W = 0.2 + 0.3 * (U**2 + V**2)  # toy true boundary

#         ax.plot_surface(
#             U, V, W,
#             alpha=0.3,
#             linewidth=0,
#             antialiased=True,
#             color="gray",
#             label="true boundary",
#         )
#         from matplotlib.lines import Line2D
#         proxy_true = Line2D([0], [0], color="gray", lw=6, alpha=0.3)
#         handles, labels = ax.get_legend_handles_labels()
#         handles.append(proxy_true)
#         ax.legend(handles, labels)
#     else:
#         ax.legend()

#     ax.set_xlabel("x1")
#     ax.set_ylabel("x2")
#     ax.set_zlabel("x3")
#     ax.set_title(title)

#     ax.set_xlim(0, 1)
#     ax.set_ylim(0, 1)
#     ax.set_zlim(0, 1)

#     if show:
#         plt.tight_layout()

In [ ]:
# plot_gp_decision_boundary_3d(
#         model2,
#         X_current,
#         y_current,
#         title=f"GPC Decision Boundary – iteration {it_print}",
#         n_grid=80,
#         eps=0.03,
#         show_true_boundary=True,
#         show=True,
#     )
# savepath = r"C:\Users\justu\OneDrive - Delft University of Technology\Documents\Master Thesis\Progress Meetings\Progress meeting 3\Figures\3D_toy_nonlinear_function_with_pool_based_search_and_mcsearchnum6000"
# # plt.savefig(Path(savepath) / 'Decision_boundary.png')

In [ ]:
def global_misclassification(
    model,
    label_func,
    n_test=100_000,
    seed=None,
):
    """
    Global misclassification metric for trail functions. 

    Parameters
    ----------
    model : Model
        The GP-based classifier.
    label_func : function
        The trial function considered. 
    n_test : int
        Number of test samples to draw uniformly in the design space.
    seed : int | None
        Random seed for reproducibility.

    Returns
    -------
    miscl_rate : float
        Fraction of misclassified test points by the prediction field of the GPC to the trial function. 
    """
    if seed is not None:
        rng = np.random.default_rng(seed)
        rand = rng.random
    else:
        rand = np.random.random

    d = model.f_num
    xlow, xhigh = model.xinterval  # these are your global bounds

    X_test = xlow + (xhigh - xlow) * rand(size=(n_test, d))

    try:
        y_true = label_func(X_test)
    except TypeError:
        if d == 2:
            y_true = label_func(X_test[:, 0], X_test[:, 1])
        elif d == 3:
            y_true = label_func(X_test[:, 0], X_test[:, 1], X_test[:, 2])
        elif d == 4:
            y_true = label_func(X_test[:, 0], X_test[:, 1], X_test[:, 2], X_test[:, 3])
        elif d == 5:
            y_true = label_func(X_test[:, 0], X_test[:, 1], X_test[:, 2], X_test[:, 3], X_test[:, 4])
        elif d == 6:
            y_true = label_func(X_test[:, 0], X_test[:, 1], X_test[:, 2], X_test[:, 3], X_test[:, 4], X_test[:, 5])
        elif d==12:
            y_true = label_func(X_test[:, 0], X_test[:, 1], X_test[:, 2], X_test[:, 3], X_test[:, 4], X_test[:, 5], X_test[:, 6], X_test[:, 7], X_test[:, 8], X_test[:, 9], X_test[:, 10], X_test[:, 11])
        else:
            raise ValueError(
                "extend this function for higher dimensions."
            )

    y_true = np.asarray(y_true, dtype=int).ravel()

    p_pred = model.predict_proba(X_test)[:, 1]  
    y_pred = (p_pred >= 0.5).astype(int)     

    miscl_rate = np.mean(y_pred != y_true)
    return miscl_rate

In [ ]:
err_global_12d = global_misclassification(
    model,
    label_func=label_toy_12d,
    n_test=1_000_000,
    seed=None,
)
print("Global misclassification (12D toy):", err_global_12d)

In [ ]:
err_global_6d = global_misclassification(
    model,
    label_func=label_toy_6d,
    n_test=1_000_000,
    seed=123,
)
print("Global misclassification (6D toy):", err_global_6d)

In [ ]:
err_global_3d = global_misclassification(
    model2,
    label_func=label_nonlinear_boundary_3d_sinus,
    n_test=600_000,
    seed=123,
)
print("Global misclassification (3D toy):", err_global_3d)

In [ ]:
err_global_2d = global_misclassification(
    model,
    label_func=label_sqrt_boundary,
    n_test=200_000,
    seed=None,
)
print("Global misclassification (2D toy):", err_global_2d)

In [ ]:
err_global_4d = global_misclassification(
    model2,
    label_func=label_nonlinear_boundary_4d,
    n_test=200_000,
    seed=121,
)
print("Global misclassification (4D toy):", err_global_4d)

In [ ]:
plt.plot(np.arange(len(boundary_frac_list)), boundary_frac_list)
plt.hlines(0.01, 0, len(boundary_frac_list), color='r', ls='--', label="Boundary spread threshold: 0.01")
plt.title('Boundary spread metric')
plt.ylabel('Boundary spread')
plt.xlabel('Iterations')
plt.grid(alpha=0.3)
plt.legend();
# plt.savefig(Path(savepath) / 'Boundary_spread.pdf')

In [ ]:
plt.plot(np.arange(len(flips_frac_list)), flips_frac_list)
plt.hlines(np.max(flips_frac_list)*0.25, 0, len(flips_frac_list), color='r', ls='--', label="Threshold hyperparameter optimisation")
plt.title("Boundary change metric")
plt.ylabel('Boundary change')
plt.xlabel("Iterations")
plt.grid(alpha=0.3)
plt.legend();
# plt.savefig(Path(savepath) / 'Boundary_change.pdf')